<a href="https://colab.research.google.com/github/EvgenyEsin/Spark_Apache/blob/main/Spark_Apache_sem_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ к семинару 4

Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=1b23ffa311f12528b602e42a4451ddab41ac71d802275c78673ebea8ffd639d6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [55]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum
from pprint import pprint
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [56]:
# Создаем SparkSession
spark = SparkSession.builder.appName("Homework4").getOrCreate()

In [57]:
# Создаем поток данных с использованием источника данных `rate`
streaming_df = spark \
.readStream \
.format("rate") \
.option("maxFilesPerTrigger", 1) \
.load()

In [58]:
# Оставляем только те строки, где значение нечетное
filtered_df = streaming_df.filter(col("value") % 2 == 1)

In [59]:
# Суммирование
sum_odds = filtered_df.agg(sum("value"))

In [60]:
# Выводим результат
query = sum_odds \
.writeStream \
.outputMode("complete") \
.queryName("Homework4") \
.format("memory") \
.start()
query.awaitTermination(5)

False

In [61]:
from time import sleep
for x in range(3):
    spark.sql("select * from Homework4").show(3)
    sleep(2)

+----------+
|sum(value)|
+----------+
|         4|
+----------+

+----------+
|sum(value)|
+----------+
|         9|
+----------+

+----------+
|sum(value)|
+----------+
|        25|
+----------+



In [62]:
query.stop()